# JSON utilities
> Functions that prepare label files and split the data into train validation and test sets

In [1]:
#hide
# default_exp json_utils
from nbdev.showdoc import *
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# hide
# hidden imports
# from  cellsegment.json_utils import *   #hide this otherwise recurseive 

In [3]:
# export
import random
from  cellsegment.core import *

import json
import shutil
import pandas
from fastai.vision import get_image_files, parallel, partial, PIL, Path, get_files
import numpy as np

from PIL import Image, ImageDraw
import os

In [4]:
# export
def createjson(fn='', offset=0):
    'create a Labelme compatable json file'
    data = {
        'version': '3.13.1',
        'flags': {},
        'shapes': [],
        'lineColor': [0, 255, 0, 255],
        'fillColor': [255, 255, 0, 20],
        'imagePath': fn,
        'offset': offset,
        'imageData': None,
        'imageHeight': 2464,
        'imageWidth': 2464}
    return data

TEST_JSON_STRING = """
{
  "version": "3.13.1",
  "flags": {},
  "shapes": [
    {
      "label": "Liver Fluke",
      "line_color": [0, 0, 255, 127],
      "fill_color": [0, 0, 255, 127],
      "points": [
        [622.4545454545455, 523.8484848484848], [642.4545454545455, 543.8484848484848]
      ],
      "shape_type": "circle",
      "flags": {}
    }
  ],
  "lineColor": [0, 255, 0, 255],
  "fillColor": [255, 255, 0, 20],
  "imagePath": "235443 - 1.jpg",
  "imageData": null,
  "imageHeight": 813,
  "imageWidth": 830,
  "offset": 0
}
"""

see lableme at https://github.com/wkentaro/labelme/blob/master/README.md

In [5]:
# export
def annotate_json(data, shape_type='circle', points=None):
    # Annotate the json file with list of points
    assert shape_type == 'point' or shape_type == 'circle' or shape_type == 'rectangle'

    def add_anno(data, item):
        cx, cy = item['point']
        pnt_list = []
        if shape_type == 'point':
            pnt_list = [[cx, cy]]
        elif shape_type == 'rectangle':  # bounding points
            r = 40
            pnt_list = [[cx - r, cy - r], [cx + r, cy + r]]
        elif shape_type == 'circle':   # center & radius
            r = 20
            pnt_list = [[cx, cy], [cx + r, cy + r]]

        probability = str(item['probability']) if 'probability' in item else str(0)

        if item['label'] == 1:
            line_color = [255, 0, 0, 127]
            data['shapes'].append({
                "label": str(item['label']), "line_color": [255, 0, 0, 127], "fill_color": [255, 0, 0, 127],
                "points": pnt_list, "shape_type": shape_type, 'probability': probability
            })

        elif item['label'] == 2:
            line_color = [0, 255, 0, 127]
            data['shapes'].append({
                "label": str(item['label']), "line_color": [0, 255, 0, 127], "fill_color": [0, 255, 0, 127],
                "points": pnt_list, "shape_type": shape_type, 'probability': probability
            })

        else:
            line_color = [0, 0, 255, 127]
            data['shapes'].append({
                "label": str(item['label']), "line_color": [0, 0, 255, 127], "fill_color": [0, 0, 255, 127],
                "points": pnt_list, "shape_type": shape_type, 'probability': probability
            })

    def add_annotations(data, points):
        for item in points:
            add_anno(data, item)

    add_annotations(data, points)
    return data

In [6]:
# export
def create_json_from_CSV(csv_fn, load_jpg_fn, offset=0, height=0, width=0):
    # Annotate the json file with predictions
    data = {
        'version': '3.13.1',
        'flags': {},
        'shapes': [],
        'lineColor': [0, 255, 0, 255],
        'fillColor': [255, 255, 0, 20],
        'imagePath': load_jpg_fn,
        'offset': offset,
        'imageData': None,
        'imageHeight': height,
        'imageWidth': width}

    # species_name = ['Nematodirus', 'Strongyle', 'other']

    egg_centers = []
    with open(csv_fn, mode='r') as f:
        line_count = 0
        for line in f:
            row, col, species = [int(x) for x in line.split(',')]
            egg_centers.append({"label": species, "particleType": species, "point": [row, col]})
            line_count += 1

    data = annotate_json(data, points=egg_centers)
    data
    return data

In [7]:
# export
def csv_to_json_dir(src_path, dest_path, number_files='all'):
    """ Convert an entire directory of Techion CSV files to JSON files. Store in the dest directory
    CSV format is x,y,species_number
    1383,1571,2
    1687,1822,1
    2036,1327,1
    """
    print('Converting an entire directory of Techion CSV files to JSON files')
    fnames_csv = sorted(get_files(src_path, extensions=['.csv']))
    fnames_jpg = sorted(get_files(src_path, extensions=['.jpg']))
    if isinstance(number_files, int):
        fnames_csv = fnames_csv[:number_files]
        fnames_jpg = fnames_jpg[:number_files]
    print('Number of csv & jpg files to convert', len(fnames_csv), len(fnames_jpg))

    Path(dest_path).mkdir(parents=True, exist_ok=True)
    print(f"src_path {src_path}")
    print(f"dest_path {dest_path}")

    for i,fn in enumerate(fnames_csv):
        parent = Path(fn).parents[0]
        name = Path(fn).name.split('.')[0]
        jpg_fn = f"{parent}/{name}.jpg"
        json_fn = f"{parent}/{name}.json"

        if Path(jpg_fn).is_file():
            img = PIL.Image.open(jpg_fn)
            data = create_json_from_CSV(fn, Path(jpg_fn).name, height=img.size[1], width=img.size[0])
            with open(json_fn, 'w') as outfile:
                # json.dump(data, outfile, indent=2)
                # print(f'Saving File {json_fn}')
                json.dump(data, outfile, ensure_ascii=False, indent=4)
                progress_bar(i+1,50)

In [8]:
# export
def resize_json_file(fn, scale=1, offset=0):
    data = json.load(open(fn))

    for s, sh in enumerate(data['shapes']):
        for p, pnt in enumerate(sh["points"]):
            data['shapes'][s]['points'][p][0] = round(data['shapes'][s]['points'][p][0] * scale)
            data['shapes'][s]['points'][p][1] = round(data['shapes'][s]['points'][p][1] * scale)
            data['shapes'][s]['points'][p][0] = data['shapes'][s]['points'][p][0] - offset
    data['imageData'] = None
    data['offset'] = offset
    data['scale'] = scale
    data['imageWidth'] = int (data['imageWidth'] * scale)
    data['imageHeight'] = int (data['imageHeight'] * scale)
    return data

In [9]:
# export
def resize_json_dir(file_data, src_path, dest_path, number_files='all', height=800):
    """
    Resize src_path directory of JSON files and store in dest_path directory
    There needs to be resized jpg files in the dest_path directory.
    An example is  226260 - 1|0.436047|221|.jpg  where |0.436047|221| marks the factor and offset factors to apply
    :param src_path: Source path where json files are
    :param dest_path: Destination path to store resized json files
    :param number_files: Number of json files to process, leave empty for all files in directory
    :return: number of file processed
    """
    df = pandas.read_csv(file_data, index_col=0)
    __number_files = df.shape[0]

    print(f'Number of JSON files: {__number_files}, Number to resize: {number_files}')
    Path(dest_path).mkdir(parents=True, exist_ok=True)
    if isinstance(number_files, int):
        __number_files = number_files

    for i in range(__number_files):
        f_stem = df.loc[i,'Name'].split('.')[0]
        scale = float(height) / df.loc[i,'Height']
        offset = 0
        data = resize_json_file(f'{src_path}/{f_stem}.json', scale=float(scale), offset=int(offset))
        data['imagePath'] = df.loc[i,'Name']
        with open(f'{dest_path}/{f_stem}.json', 'w') as outfile:
            json.dump(data, outfile, ensure_ascii=False, indent=4)

        progress_bar(i + 1, 50)
    print('')
    print(i+1, ' json files processed')
    return i+1

> Test create_json_from_CSV()
>

In [10]:
from json import JSONEncoder
json_object = create_json_from_CSV('nbs_test_data/235443 - 1.jpg.points.csv', 'nbs_test_data/235443 - 1.jpg')

b = prettyjson(json_object)
print(b)

NameError: name 'prettyjson' is not defined

In [ ]:
# export
# https://stackoverflow.com/a/56497521/104668

def prettyjson(obj, indent=2, maxlinelength=80):
    """
    Renders JSON content with indentation and line splits/concatenations to fit maxlinelength.
    Only dicts, lists and basic types are supported
    - https://github.com/andy-gh/prettyjson
    - https://stackoverflow.com/a/56497521/104668
    """

    items, _ = getsubitems(obj, itemkey="", islast=True, maxlinelength=maxlinelength, level=0)
    return indentitems(items, indent, level=0)

In [ ]:
#hide
#export
import types
def getsubitems(obj, itemkey, islast, maxlinelength, level):
    items = []
    is_inline = True      # at first, assume we can concatenate the inner tokens into one line

    isdict = isinstance(obj, dict)
    islist = isinstance(obj, list)
    istuple = isinstance(obj, tuple)
    isbasictype = not (isdict or islist or istuple)

    # build json content as a list of strings or child lists
    if isbasictype:
        # render basic type
        keyseparator  = "" if itemkey == "" else ": "
        itemseparator = "" if islast else ","
        items.append(itemkey + keyseparator + basictype2str(obj) + itemseparator)

    else:
        # render lists/dicts/tuples
        if isdict:    opening, closing, keys = ("{", "}", iter(obj.keys()))
        elif islist:  opening, closing, keys = ("[", "]", range(0, len(obj)))
        elif istuple: opening, closing, keys = ("[", "]", range(0, len(obj)))    # tuples are converted into json arrays

        if itemkey != "": opening = itemkey + ": " + opening
        if not islast: closing += ","

        count = 0
        itemkey = ""
        subitems = []

        # get the list of inner tokens
        for (i, k) in enumerate(keys):
            islast_ = i == len(obj)-1
            itemkey_ = ""
            if isdict: itemkey_ = basictype2str(k)
            inner, is_inner_inline = getsubitems(obj[k], itemkey_, islast_, maxlinelength, level+1)
            subitems.extend(inner)                        # inner can be a string or a list
            is_inline = is_inline and is_inner_inline     # if a child couldn't be rendered inline, then we are not able either

        # fit inner tokens into one or multiple lines, each no longer than maxlinelength
        if is_inline:
            multiline = True

            # in Multi-line mode items of a list/dict/tuple can be rendered in multiple lines if they don't fit on one.
            # suitable for large lists holding data that's not manually editable.

            # in Single-line mode items are rendered inline if all fit in one line, otherwise each is rendered in a separate line.
            # suitable for smaller lists or dicts where manual editing of individual items is preferred.

            # this logic may need to be customized based on visualization requirements:
            if (isdict): multiline = False
            if (islist): multiline = True

            if (multiline):
                lines = []
                current_line = ""
                current_index = 0

                for (i, item) in enumerate(subitems):
                    item_text = item
                    if i < len(inner)-1: item_text = item + ","

                    if len (current_line) > 0:
                        try_inline = current_line + " " + item_text
                    else:
                        try_inline = item_text

                    if (len(try_inline) > maxlinelength):
                        # push the current line to the list if maxlinelength is reached
                        if len(current_line) > 0: lines.append(current_line)
                        current_line = item_text
                    else:
                        # keep fitting all to one line if still below maxlinelength
                        current_line = try_inline

                    # Push the remainder of the content if end of list is reached
                    if (i == len (subitems)-1): lines.append(current_line)

                subitems = lines
                if len(subitems) > 1: is_inline = False
            else: # single-line mode
                totallength = len(subitems)-1   # spaces between items
                for item in subitems: totallength += len(item)
                if (totallength <= maxlinelength):
                    str = ""
                    for item in subitems: str += item + " "  # insert space between items, comma is already there
                    subitems = [ str.strip() ]               # wrap concatenated content in a new list
                else:
                    is_inline = False


        # attempt to render the outer brackets + inner tokens in one line
        if is_inline:
            item_text = ""
            if len(subitems) > 0: item_text = subitems[0]
            if len(opening) + len(item_text) + len(closing) <= maxlinelength:
                items.append(opening + item_text + closing)
            else:
                is_inline = False

        # if inner tokens are rendered in multiple lines already, then the outer brackets remain in separate lines
        if not is_inline:
            items.append(opening)       # opening brackets
            items.append(subitems)      # Append children to parent list as a nested list
            items.append(closing)       # closing brackets

    return items, is_inline


def basictype2str(obj):
    if isinstance (obj, str):
        strobj = "\"" + str(obj) + "\""
    elif isinstance(obj, bool):
        strobj = { True: "true", False: "false" }[obj]
    else:
        strobj = str(obj)
    return strobj


def indentitems(items, indent, level):
    """Recursively traverses the list of json lines, adds indentation based on the current depth"""
    res = ""
    indentstr = " " * (indent * level)
    for (i, item) in enumerate(items):
        if isinstance(item, list):
            res += indentitems(item, indent, level+1)
        else:
            islast = (i==len(items)-1)
            # no new line character after the last rendered line
            if level==0 and islast:
                res += indentstr + item
            else:
                res += indentstr + item + "\n"
    return res